vit-gpt2-image-captioning

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import pipeline
from tqdm.auto import tqdm
import pandas as pd
import os
import torch
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device = -1)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
url = 'https://www.unigreet.com/wp-content/uploads/2022/11/100-very-special-good-morning-images-quotes-photos.jpg'
image_to_text(url)

In [5]:
def get_caption(image_to_text, url):
    try:
        return image_to_text(url)[0]['generated_text']
    except:
        return '#'

Gathering images from instaData

In [ ]:
import os

In [ ]:
def read_caption_file(f):
    s = ''
    for i in f.readlines():
        s+=i
    return s

In [ ]:
images = []
base_path = '/content/drive/MyDrive/Information Retrieval Project/InstaData'
for i in os.listdir(base_path):
    path = base_path + '/' + i
    if not os.path.isdir(path):
        continue    
    for j in os.listdir(base_path + '/' + i):
        if 'jpg' in j and 'profile_pic' not in j:
            images.append(path + '/' + j)

In [ ]:
base_path = '/content/drive/MyDrive/Information Retrieval Project/InstaData'
img_caption = {}
for i in tqdm(images):
    root_name = i[:i.index('UTC') + 3]
    try:
        c = open(root_name + '.txt', 'r')
        img_caption[i] = read_caption_file(c)
    except:
        continue

100%|██████████| 3500/3500 [00:47<00:00, 74.25it/s]


In [ ]:
df = pd.DataFrame(list(img_caption.items()), columns = ['Image', 'Real_insta_caption'])
df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/InstaData/data.pkl')

Adding the generated descriptive caption to the dataframe

In [ ]:
tqdm.pandas()
df['Generated_descriptive_caption'] = df.progress_apply(lambda x: get_caption(image_to_text, x['Image']), axis = 1)

  0%|          | 0/3468 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/InstaData/data_with_generated_captions.pkl')

In [ ]:
df

Gathering images from newInstaData

In [ ]:
images = []
base_path = '/content/drive/MyDrive/Information Retrieval Project/newInstaData'
for i in os.listdir(base_path):
    path = base_path + '/' + i
    if not os.path.isdir(path):
        continue    
    for j in os.listdir(base_path + '/' + i):
        if 'jpg' in j and 'profile_pic' not in j:
            images.append(path + '/' + j)

In [ ]:
base_path = '/content/drive/MyDrive/Information Retrieval Project/newInstaData'
img_caption = {}
for i in tqdm(images):
    root_name = i[:i.index('UTC') + 3]
    try:
        c = open(root_name + '.txt', 'r')
        img_caption[i] = read_caption_file(c)
    except:
        continue

  0%|          | 0/5549 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(list(img_caption.items()), columns = ['Image', 'Real_insta_caption'])
# df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/newInstaData/data.pkl')
df.isna().any()

Image                 False
Real_insta_caption    False
dtype: bool

In [ ]:
df.head()

In [ ]:
tqdm.pandas()
df['Generated_descriptive_caption'] = df.progress_apply(lambda x: get_caption(image_to_text, x['Image']), axis = 1)

In [ ]:
df.iloc[5000:]

In [ ]:
df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/newInstaData/data_with_generated_captions.pkl')

In [ ]:
df_temp = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/newInstaData/data_with_generated_captions.pkl')

In [ ]:
df_temp

In [ ]:
from PIL import Image
Image.open('/content/drive/MyDrive/Information Retrieval Project/newInstaData/thestylestalkercom/2023-03-03_20-34-52_UTC_4.jpg')

Conceptual Captions

In [ ]:
cc = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/Image captioning datasets/Pickle files/conceptual.pkl')
cc = cc.iloc[1000:2000]

In [ ]:
cc

In [ ]:
tqdm.pandas()
cc['Generated_descriptive_caption'] = cc.progress_apply(lambda x: get_caption(image_to_text, x['Image']), axis = 1)

In [ ]:
cc

In [69]:
conceptual_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/conceptual_results/data_with_generated_captions.pkl')

EVAL METRICS

In [74]:
def blue_score(reference, caption): 
    # reference = reference.split()
    # caption = caption.split()   
    try:
        return sentence_bleu(reference, caption)
    except:
        return 0

In [ ]:
blue_score('sparkling glittery gold uppercase or capital letter l in a 3d illustration with a bright golden color and rough chiseled metallic surface jagged font isolated on a white background with clipping path .', 'a white and black vase with a bird on it')

In [ ]:
cc = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/conceptual_results/data_with_generated_captions.pkl')

In [ ]:
tqdm.pandas()

In [67]:
conceptual_df['bleu_score'] = conceptual_df.progress_apply(lambda x: blue_score(x['Original Caption'], x['Generated_descriptive_caption']), axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
cc

COCO

In [ ]:
coco = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/Image captioning datasets/Pickle files/coco.pkl')

In [ ]:
coco_list = [coco[i] for i in coco.keys()]

In [ ]:
coco_df = pd.DataFrame(coco_list)
coco_df.drop(columns = ['segmentation', 'bbox', 'category_id'], inplace = True)

In [ ]:
coco_df = coco_df.iloc[:1000]

In [ ]:
coco_df

In [ ]:
tqdm.pandas()
coco_df['Generated_descriptive_caption'] = coco_df.progress_apply(lambda x: get_caption(image_to_text, x['url']), axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
<ipython-input-40-de3bd8bbf133>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coco_df['Generated_descriptive_caption'] = coco_df.progress_apply(lambda x: get_caption(image_to_text, x['url']), axis = 1)


In [ ]:
coco_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/coco_results/data_with_generated_captions.pkl')

In [ ]:
coco_df.head()

,url,captions,Generated_descriptive_caption,bleu_score
0,http://images.cocodataset.org/val2014/COCO_val...,[A man with a red helmet on a small moped on a...,a man riding a motorcycle on a dirt road,0.986925
1,http://images.cocodataset.org/val2014/COCO_val...,[A woman wearing a net on her head cutting a c...,a woman cutting a cake with a knife,0.810337
2,http://images.cocodataset.org/val2014/COCO_val...,[A child holding a flowered umbrella and petti...,a young boy holding an umbrella while standing...,0.890771
3,http://images.cocodataset.org/val2014/COCO_val...,[A young boy standing in front of a computer k...,a young boy is using a computer while sitting ...,0.714911
4,http://images.cocodataset.org/val2014/COCO_val...,[a boy wearing headphones using one computer i...,a boy in a black shirt and a black and white c...,0.468980


In [ ]:
coco_df['bleu_score'] = coco_df.progress_apply(lambda x: blue_score(x['captions'], x['Generated_descriptive_caption']), axis = 1)

In [65]:
coco_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/coco_results/data_with_generated_captions_and_bleu.pkl')

In [ ]:
coco_df['bleu_score'].mean()

0.7445000877202034

FLICKr

In [6]:
import pandas as pd
flickr_temp = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/results (1).csv')
flickr_temp = flickr_temp.drop(columns = flickr_temp.columns[1:])
flickr_temp.head()

<ipython-input-6-3565be035e9f>:2: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  flickr_temp = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/results (1).csv')


,image_name| comment_number| comment
0,1000092795.jpg| 0| Two young guys with shaggy ...
1,1000092795.jpg| 1| Two young
2,1000092795.jpg| 2| Two men in green shirts are...
3,1000092795.jpg| 3| A man in a blue shirt stand...
4,1000092795.jpg| 4| Two friends enjoy time spen...


In [7]:
lst = []
for i in flickr_temp['image_name| comment_number| comment']:
    temp = []
    content = i.split('|')
    try:
        temp.append('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/flickr30k_images/flickr30k_images/' + content[0])
        temp.append(content[2])     
        lst.append(temp)
    except:
        continue

In [14]:
lst[0]

['/content/drive/MyDrive/Information Retrieval Project/flickr dataset/flickr30k_images/flickr30k_images/1000092795.jpg',
 ' Two young guys with shaggy hair look at their hands while hanging out in the yard .']

In [9]:
flickr_df = pd.DataFrame(lst, columns = ['Image', 'Captions']).iloc[:1000]
flickr_df

,Image,Captions
0,/content/drive/MyDrive/Information Retrieval P...,Two young guys with shaggy hair look at their...
1,/content/drive/MyDrive/Information Retrieval P...,Two young
2,/content/drive/MyDrive/Information Retrieval P...,Two men in green shirts are standing in a yard .
3,/content/drive/MyDrive/Information Retrieval P...,A man in a blue shirt standing in a garden .
4,/content/drive/MyDrive/Information Retrieval P...,Two friends enjoy time spent together .
...,...,...
995,/content/drive/MyDrive/Information Retrieval P...,A man lies on a mat in a parking lot between ...
996,/content/drive/MyDrive/Information Retrieval P...,A shirtless man is laying down in the middle ...
997,/content/drive/MyDrive/Information Retrieval P...,A man laying down in middle of street during ...
998,/content/drive/MyDrive/Information Retrieval P...,A man in blue shorts lays down outside in a p...


In [10]:
tqdm.pandas()
flickr_df['Generated_descriptive_caption'] = flickr_df.progress_apply(lambda x: get_caption(image_to_text, x['Image']), axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [76]:
flickr_df['bleu_score'] = flickr_df.progress_apply(lambda x: blue_score(x['Captions'], x['Generated_descriptive_caption']), axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1.3648707879917412e-231

In [64]:
flickr_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/flickr_results/data_with_generated_captions_and_bleu.pkl')

ROUGE

In [18]:
pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=5b6130e845f1cb36cdf51238475241c6f869136c7aba81e2faecddc77bdcf902
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [24]:
from rouge_score import rouge_scorer
def rouge_score(reference, caption):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference,caption)                        
    return scores['rougeL'].precision

METEOR

In [56]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def meteor_score(refs, hypos):
    refs = [refs]
    return meteor_score(refs, hypos)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
meteor_score('hi my name i karan', 'bye my name') 

READING FINAL DFs

In [23]:
import pandas as pd
coco_df = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/coco_results/data_with_generated_captions_and_bleu.pkl')
conceptual_df = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/conceptual_results/data_with_generated_captions.pkl')
insta_df = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/newInstaData/data_with_generated_captions.pkl')
flickr_df = pd.read_pickle('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/flickr_results/data_with_generated_captions_and_bleu.pkl')

In [36]:
coco_df['rouge_score'] = coco_df.progress_apply(lambda x: rouge_score(x['captions'][0], x['Generated_descriptive_caption']), axis = 1)
conceptual_df['rouge_score'] = conceptual_df.progress_apply(lambda x: rouge_score(x['Original Caption'], x['Generated_descriptive_caption']), axis = 1)
insta_df['rouge_score'] = insta_df.progress_apply(lambda x: rouge_score(x['Real_insta_caption'], x['Generated_descriptive_caption']), axis = 1)
flickr_df['rouge_score'] = flickr_df.progress_apply(lambda x: rouge_score(x['Captions'], x['Generated_descriptive_caption']), axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/5528 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
insta_df['bleu_score'] = insta_df.progress_apply(lambda x: blue_score(x['Real_insta_caption'], x['Generated_descriptive_caption']), axis = 1)

In [ ]:
coco_df.head()

In [ ]:
conceptual_df.head()

In [ ]:
insta_df.head()

In [ ]:
flickr_df.head()

In [82]:
coco_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/FINAL SUBMISSION FILES/coco_df.pkl')
conceptual_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/FINAL SUBMISSION FILES/conceptual_df.pkl')
insta_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/FINAL SUBMISSION FILES/insta_df.pkl')
flickr_df.to_pickle('/content/drive/MyDrive/Information Retrieval Project/FINAL SUBMISSION FILES/flickr_df.pkl')